In [31]:
import os, csv
import json
import pandas as pd

import spotipy.util as util
import requests

In [2]:
with open('settings/webbify_credentials.txt', 'r') as tfile:
    fileContents = tfile.read().split('\n')
    CLIENT_ID = fileContents[0].split(':')[1][1:]
    CLIENT_SECRET = fileContents[1].split(':')[1][1:]

In [3]:
USERNAME = 'nghokui@gmail.com'
TOKEN_CACHE = '.cache-{}'.format(USERNAME)
if os.path.exists(TOKEN_CACHE):
    os.remove(TOKEN_CACHE)

    
AUTH_ARGS = {
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
    'redirect_uri': 'https://localhost:8888/',
    'scope': 'user-read-recently-played'
}
token = util.prompt_for_user_token(USERNAME, **AUTH_ARGS)



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opening https://accounts.spotify.com/authorize?redirect_uri=https%3A%2F%2Flocalhost%3A8888%2F&client_id=d5eba668c908462aa3183670fe307302&scope=user-read-recently-played&response_type=code in your browser


Enter the URL you were redirected to: https://localhost:8888/?code=AQA8mWlEtyrDBQZ4Mi3zheb4L1CmoABIjXJ6fQTPUz0fCoyygoj-OpgwYo0K1liKQKXCMD0sbcyWqzeeXR-OYKY2IoyfsQPKsRK_lwKpqmw3MfhacpShBaQR4CbpQXSs5w9jZ4Ey-aSaFl1NbIxdLW1uGL1UmmZgf1gzyxOO1kI10gkHTWbQRGSMBB9vCSVDKgLvFf7RNfFpKM9B_ZdDU49otLT0DKuE




In [4]:
URL_BASE = 'https://api.spotify.com'
HEADERS = {'Authorization': 'Bearer {}'.format(token)}

data = requests.get('{}/v1/me'.format(URL_BASE), headers=HEADERS)
data.json()

{'display_name': 'Brian Ng',
 'external_urls': {'spotify': 'https://open.spotify.com/user/nghokui'},
 'followers': {'href': None, 'total': 20},
 'href': 'https://api.spotify.com/v1/users/nghokui',
 'id': 'nghokui',
 'images': [{'height': None,
   'url': 'https://scontent.xx.fbcdn.net/v/t31.0-1/c53.0.320.320a/p320x320/19956194_3497275234192_4181604532212724762_o.jpg?_nc_cat=107&_nc_oc=AQmEaKZZqmhvaRCa3S9ZmQLnxnbUIb5w1kCaBitsc_FIweIOjOa0W0RxoDJ3NqrEDjA&_nc_ht=scontent.xx&_nc_tp=1&oh=43c43280eaa41d901d1313468e77fdf2&oe=5EA63217',
   'width': None}],
 'type': 'user',
 'uri': 'spotify:user:nghokui'}

In [8]:
PLAYLIST_URL_BASE = 'https://api.spotify.com/v1/playlists/{}'
PLAYLIST_URI = '2PTEg5tENIOnT8b09Bb8IJ'

TRACK_KEYS = {
    'popularity': 'trackPopularity',
    'duration_ms': 'trackDurationMS',
    'explicit': 'trackExplicit',
    'name': 'trackName',
    'id': 'trackID'
}
ALBUM_KEYS = {
    'album_type': 'albumType',
    'release_date_precision': 'albumReleaseDatePrecision',
    'total_tracks': 'albumTotalTracks',
    'name': 'albumName',
    'id': 'albumID',
    'release_date': 'albumReleaseDate'
}

In [11]:
def getPlaylistData(plistData, counterStart):
    tData, counter = [], counterStart
    for trackCt,i in enumerate(plistData['items']):
        trackData = {
            'added_at': i['added_at'],
            'added_by': i['added_by']['id']
        }

        for tKey,tVal in i['track'].items():
            trackKey = TRACK_KEYS.get(tKey, None)
            if trackKey: trackData[trackKey] = tVal

        for albKey,albVal in i['track']['album'].items():
            albumKey = ALBUM_KEYS.get(albKey, None)
            if albumKey: trackData[albumKey] = albVal

        artistData = []
        for artCt,artVal in enumerate(i['track']['artists']):
            artistTrackData = trackData.copy()
            artistTrackData['artistCt'] = artCt
            artistTrackData['artistID'] = artVal['id']
            artistTrackData['artistName'] = artVal['name']

            tData.append(artistTrackData)
        counter = counter + 1
        
    return (counter,tData)

In [12]:
firstPass = True
#nextURL = plistDataRaw['tracks']['next']

spotifyRequest = requests.get(PLAYLIST_URL_BASE.format(PLAYLIST_URI), headers=HEADERS)

if str(spotifyRequest) == '<Response [200]>':
    rawSpotifyPlistData = spotifyRequest.json()['tracks']
else:
    rawSpotifyPlistData = None

print(spotifyRequest)
data,ct = [],1
while rawSpotifyPlistData:
    retrievedData = getPlaylistData(rawSpotifyPlistData, ct)
    ct = retrievedData[0]
    data = data + retrievedData[1]
    
    
    if rawSpotifyPlistData['next']:
        nextURI = rawSpotifyPlistData['next']
        nextData = requests.get(nextURI, headers=HEADERS)
        rawSpotifyPlistData = nextData.json()
    else:
        rawSpotifyPlistData = None
        
print('Queried {} unique tracks from playlist.'.format(ct))

<Response [200]>
Queried 682 unique tracks from playlist.


In [32]:
keys = list(data[0].keys())
tabularData = [['num'] + keys]

for i,item in enumerate(data):
    lineData = [item[j] for j in keys]
    lineData = [i+1] + lineData
    
    tabularData.append(lineData)
df = pd.DataFrame(tabularData)
df.columns = df.iloc[0]
df = df.drop(0, axis=0)


# write it to csv
fn = 'data/ThingsPlaylist.csv'
df.to_csv(fn, sep='|', quoting=csv.QUOTE_NONNUMERIC, quotechar='`')

In [ ]:
for item in data[-81:]:
    print(item['playlistNo'])